In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import os
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import cftime

In [3]:
ds = xr.open_dataset('/glade/work/dmeidan/WACCM_forcast_files/Daphne-emission_x3/emissions-fcst_bc_a4_bb_surface_0.9x1.25.nc')

In [4]:
ds

<xarray.Dataset>
Dimensions:  (time: 1096, lat: 192, lon: 288)
Coordinates:
  * lon      (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat      (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * time     (time) datetime64[ns] 2021-01-01 2021-01-02 ... 2024-01-01
Data variables:
    bb       (time, lat, lon) float32 ...
    date     (time) int32 ...
Attributes:
    data_title:     FINN1.5 and FINNv1-NRT fire emissions for forecasting
    data_creator:   Louisa Emmons (emmons@ucar.edu)
    creation_date:  Thu Jul  1 10:08:55 2021
    history:        Sat Jul  1 09:29:48 2023: ncap2 -s bb=bb*3 emissions-fcst...
    NCO:            netCDF Operators version 5.1.4 (Homepage = http://nco.sf....

In [21]:
ds2 = ds.sel(time=slice('2023-01-01', '2023-12-31'))

In [36]:
len(ds.lat)
a = np.ones((4, 5))
b = np.zeros((5, 4, 5))

In [37]:
b[:] =a
b

array([[[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]])

In [3]:
para = xr.open_dataset('/glade/p/cesmdata/cseg/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112.nc')
para['min_NH_planting_date']

<xarray.DataArray 'min_NH_planting_date' (pft: 79)>
[79 values with dtype=float64]
Coordinates:
    pftname  (pft) |S40 ...
Dimensions without coordinates: pft
Attributes:
    long_name:  Minimum planting date for the Northern Hemipsphere
    units:      YYYYMMDD
    comment:    Typical U.S. earliest planting dates according to AgroIBIS: M...

In [4]:
para['min_NH_planting_date'].values

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan, 401., 401., 401., 401., 901.,
       901., 501., 501., 401., 401., 901., 901., 401., 401., 901., 901.,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan, 401., 401.,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan, 101., 101.,  nan,  nan,  nan,
        nan, 101., 101.,  nan,  nan, 401., 401., 401., 401., 320., 320.,
       415., 415.])